In [4]:
import torchvision.transforms as transforms
import numpy as np
import skimage.io as io
from sklearn.metrics import mean_squared_error
from skimage.color import rgb2gray
from skimage.metrics import structural_similarity as ssim
from glob import glob
import os
transform = transforms.ToTensor()
import lpips
from pathlib import Path
import torch
import scipy.io
loss_fn_alex = lpips.LPIPS(net='alex') 

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\anmol\anaconda3\lib\site-packages\lpips\weights\v0.1\alex.pth


In [44]:
from scipy import stats

In [20]:
hw = scipy.io.loadmat("hw5.mat")
blur_dmos = np.array(hw['blur_dmos']).ravel()
blur_orgs = np.array(hw['blur_orgs']).ravel()
refnames_blur = np.array(hw['refnames_blur']).ravel()

In [34]:
def file_handling(blur_dmos , blur_orgs , refnames_blur ):
 reference_img = []
 distortion_img = []
 index = 0
 DMOS = []
 files_path =sorted(glob("gblur/*.bmp"), key = lambda x   : int(os.path.splitext(os.path.basename(x))[0][3:]))
 for f in files_path:
    if(blur_orgs[index]==0):
      ref_img = refnames_blur[index][0]
      ref_path = Path('refimgs/'+ str(ref_img))
      ref_img = io.imread(ref_path)
      dis_img = io.imread(f)
      reference_img.append(rgb2gray(ref_img))
      distortion_img.append(rgb2gray(dis_img))
      DMOS.append(blur_dmos[index])
      index +=1      
    else :
      index += 1
      continue
 return np.array(reference_img) , np.array(distortion_img) , np.array(DMOS)

In [50]:
def Tensor_detach(LPIPS_Ind):
 st_l =[]
 for i in range(145):
        st=str(LPIPS_Ind[i][0][0][0][0])
        st= st[7:13]
        st_l.append(float(st))
 LPIPS_Ind = np.array(st_l)
 return LPIPS_Ind


In [54]:
reference_imgs , distortion_imgs , DMOS_vals = file_handling(blur_dmos , blur_orgs , refnames_blur )
MSE =        []
SSIM_Index = []
LPIPS_Ind =  []
for k in range(145): 
      img1 = reference_imgs[k]
      img2 = distortion_imgs[k]
      mean_square_error = mean_squared_error(img1 , img2)
      SSIM              = ssim(img1 , img2, data_range=255)
      img1 = img1.astype(np.float32)
      img2 = img2.astype(np.float32)
      LPIPS = loss_fn_alex(transform(img1), transform(img2))
      MSE.append(mean_square_error)
      SSIM_Index.append(SSIM)
      LPIPS_Ind.append(LPIPS)
    

<ipython-input-34-95b8a5907cf4>:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(reference_img) , np.array(distortion_img) , np.array(DMOS)


In [56]:
LPIPS = Tensor_detach(LPIPS_Ind)
print("Scores")
coeff1 , p1 = stats.spearmanr(np.array(DMOS_vals), np.array(MSE))
coeff2 , p2 = stats.spearmanr(np.array(DMOS_vals), np.array(SSIM_Index))
coeff3 , p3 = stats.spearmanr(np.array(DMOS_vals), LPIPS)
print(coeff1)
print(coeff2)
print(coeff3)

Scores
0.7833175877814516
-0.8552157140607777
0.9599234763240657
